In [72]:
#---------------------------------------- Importa las librerias a utilizar ----------------------------------------#
import requests
from bs4 import BeautifulSoup
import re
from selenium import webdriver
import time
from datetime import datetime

In [2]:
#---------------------------------------- Funciones complementarias ----------------------------------------#
def encuentra_batcher_png(texto):
    """Función para despejar el nombre de la imagen colocada en batcher."""
    patron = re.compile(".png")
    patron2 = re.compile("/")

    s2 = patron2.split(texto)

    for txt in s2:
        s = patron.search(txt)
        if type(s) == re.Match:
            txt2 = patron.split(txt)
            return(txt2[0])

In [3]:
#---------------------------------------- Carga todos los items de la pag ----------------------------------------#

# browser = webdriver.PhantomJS()
browser = webdriver.Chrome("C:\chromedriver_win32\chromedriver.exe")
browser.get("https://supermercado.carrefour.com.ar/almacen.html")

SCROLL_PAUSE_TIME = 0.5

# Get scroll height
last_height = browser.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height


for i in range(60):
    time.sleep(1)     #Explicit wait
    # click radio button
    try:
        python_button = browser.find_elements_by_xpath("/html/body/div[1]/div[1]/div[1]/div[8]/div/div/div[4]/a")[0]
        browser.execute_script("arguments[0].click();", python_button)
    except:
        pass
time.sleep(1)     #Explicit wait



In [66]:
#---------------------------------------- Código scrap de la página cargada ----------------------------------------#

soup = BeautifulSoup(browser.page_source,"html.parser",multi_valued_attributes=None)
all = soup.findAll("div", {"class": "col s12 product-card product-card-food"})

l=[]
for item in all:
    p={}
    info_producto = item.find("div",{"class":"producto-info"})
    p['ruta_categoria']=info_producto['data-categorytext']
    p['precio_bruto']=info_producto['data-price'].replace(",","").replace(".",",")
    if item.find("div",{"class":"batches"}) is not None:
        p['etiqueta_png']=encuentra_batcher_png(item.find("div",{"class":"batches"})['style'])
    p['precio_medida'],p['medida']=item.find("div",{"class":"precio-medida"}).text.strip().split('x')
    p['precio_medida'] = p['precio_medida'].replace("($","").strip().replace(".","")
    p['medida'] = p['medida'].replace(".)","").strip()
    try:
        p['oferta']=item.find("p",{"class","offer"}).text
    except:
        p['oferta']=None
    try:
        p['precio_oferta']=item.find("span",{"class","precio-numero"}).text.strip().replace("$","")
    except:
        p['precio_oferta']=None
    try:
        p['texto_oferta']=item.find("span",{"class","precio-texto"}).text.strip()
    except:
        p['texto_oferta']=None
    p['marca']=item.find("p",{"class","brand truncate"}).text.strip()
    p['producto']=item.find("p",{"class":"title title-food truncate"}).text.strip()
    l.append(p)
    
# browser.close()

In [62]:
l[1253]

{'ruta_categoria': 'Almacén/Sal, aderezos y saborizadores/Saborizadores',
 'precio_bruto': '78,00',
 'precio_medida': '1368,42',
 'medida': '1 K',
 'oferta': None,
 'precio_oferta': None,
 'precio_texto': None,
 'marca': 'Knorr',
 'producto': 'Saborizador Knorr albahaca y ajo 6 u.'}

In [67]:
import pandas as pd
df = pd.DataFrame(l)

In [73]:
df['fecha'] = datetime.now()

In [74]:
df.to_csv("20200426_carrefour_almacen.csv",encoding="utf-8")